# Sprint 1 - Webinar 3: Limpieza básica en Google Sheets (sin fórmulas concatenadas)


<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w1s2_1.png" width="420">
</div>

## Introducción

En esta sesión nos enfocamos en el **“día 2” del Data Analyst en spreadsheets**: **arreglar lo mínimo indispensable del dataset** para que sea **usable** (sumas/promedios correctos, categorías consistentes, texto limpio) y para que cualquier persona del equipo pueda **entender qué cambió y por qué**.

**Reglas de esta sesión (intencionales):**
- ✅ Usamos **Buscar y Reemplazar** como primera opción (herramienta, no fórmula).
- ✅ Usamos **funciones simples y aisladas**: `LOWER`, `UPPER`, `TRIM`, `NOMBREPROPIO` (`PROPER`), `SUM`, `AVERAGE`, `COUNT`, `UNIQUE`, `SUMIF`.
- ✅ Iremos **columna por columna**, con cambios directos y verificaciones rápidas.
- ❌ No usamos `SUBSTITUTE` (ni `SUSTITUIR`).
- ❌ No usamos **funciones concatenadas** (ej. `TRIM(LOWER(...))`).
- ❌ No hacemos transformaciones “sofisticadas” (CTEs, scripts, regex en fórmulas, etc.). La meta es **fundamento + hábito**.

> Idea clave: una limpieza “mínima pero bien hecha” desbloquea el 80% del análisis.

---

## Objetivos de aprendizaje

Al finalizar la sesión podrás:

1. Explicar **qué es limpiar datos** en un contexto real (y qué *no* es).
2. Aplicar **Buscar y Reemplazar** para estandarizar separadores, símbolos y textos repetidos.
3. Usar funciones básicas (`TRIM`, `LOWER`, `UPPER`, `NOMBREPROPIO`) para normalizar texto **sin concatenarlas**.
4. Validar rápidamente resultados con `COUNT`, `UNIQUE`, `SUM`, `AVERAGE` y `SUMIF`.
5. Documentar transformaciones en un **change log** para trazabilidad.

---

## Agenda (100 min)

| Minutos | Bloque | Contenido |
|---:|---|---|
| 0–10 | Apertura | Objetivo, reglas y setup del archivo |
| 10–25 | (1) Qué es “limpieza mínima” | Tipos de problemas y criterio DA |
| 25–40 | (2) Buscar y Reemplazar | Cuándo usarlo, opciones y buenas prácticas |
| 40–55 | (3) Normalización de texto | `TRIM`, mayúsculas/minúsculas, nombre propio |
| 55–90 | (4) Limpieza guiada por columnas | Checklist + verificación rápida con funciones |
| 90–100 | Cierre | Takeaways + lista de verificación para la práctica |

---

## Material de la sesión

**Dataset:** `sp1_w1_taylor_swift_da_intro.xlsx` (el mismo de la sesión anterior)

**Pestañas esperadas en Google Sheets:**
- `raw_songs`
- `raw_albums`
- `data_dictionary`
- `change_log`
- `summary_template`

> Si tu archivo no tiene exactamente esos nombres, no pasa nada: lo importante es el **rol** de cada pestaña (datos / diccionario / registro de cambios / resumen).

---

## 0) Setup en Google Sheets (10 min)

### 0.1 Importar el archivo
1. Abre Google Drive → **Nuevo** → **Hojas de cálculo de Google**.
2. Archivo → **Importar** → **Subir** → selecciona `sp1_w1_taylor_swift_da_intro.xlsx`.
3. Elige: **“Insertar hoja(s) nueva(s)”** (para mantener pestañas separadas).

### 0.2 Crear una copia “segura”
- Duplica `raw_songs` → renómbrala `songs_work`.
- Duplica `raw_albums` → renómbrala `albums_work`.

> Regla DA: **nunca** limpies sobre el “raw” sin copia. Si algo sale mal, puedes comparar.

### 0.3 Preparar el change log
En `change_log`, crea estas columnas si no existen:
- `timestamp`
- `sheet`
- `column`
- `issue`
- `action`
- `before_example`
- `after_example`
- `owner`

---

## 1) Qué significa “limpieza mínima” (15 min)

La limpieza mínima busca **hacer el dataset operable** sin rediseñarlo.

### Problemas típicos que SÍ resolvemos hoy
- Espacios extra: `"  Blank Space "` → `"Blank Space"` (`TRIM`)
- Variaciones de mayúsculas: `"taylor swift"` vs `"Taylor Swift"` (`NOMBREPROPIO`)
- Separadores inconsistentes: `"A, B / C"` → `"A; B; C"` (Buscar y Reemplazar)
- Símbolos que rompen números: `$`, `%`, `,` como separador de miles (Buscar y Reemplazar)
- Valores “texto” que deberían ser números (verificar y corregir con limpieza directa)

### Problemas que NO resolvemos hoy (a propósito)
- Reglas complejas de parsing (ej. partir texto en múltiples columnas)
- Normalización avanzada (ej. modelos relacionales perfectos)
- Deducción automatizada de errores (ej. scripts)

> Criterio: si el problema impide **sumar/promediar/contar** o **categorizar consistentemente**, entra en limpieza mínima.

---

## 2) Buscar y Reemplazar (15 min)

Esta herramienta es el “destornillador” del DA en spreadsheets: rápida, directa y controlada.

### 2.1 Dónde está
Editar → **Buscar y reemplazar**

### 2.2 Opciones importantes (úsalas con intención)
- **Buscar** en: hoja actual / todas las hojas.
- **Coincidir mayúsculas/minúsculas** (cuando sea crítico).
- **Buscar dentro de fórmulas** (normalmente NO para esta sesión).
- **Usar expresiones regulares** (opcional; úsala solo si la explicas y la pruebas en 2–3 celdas primero).

### 2.3 Buenas prácticas (DA)
1. Antes de reemplazar, usa **Buscar** y navega 5–10 coincidencias.
2. Haz el reemplazo primero en una sola hoja (`songs_work`) y valida.
3. Registra el cambio en `change_log` (qué, por qué, ejemplo antes/después).
4. Evita reemplazos “globales” sin revisar: pueden romper nombres o categorías.

### Ejemplos típicos (según el dataset)
- Reemplazar `", "` por `"; "` en columnas de listas (si aplica).
- Eliminar símbolos como `$` o `%` en columnas numéricas.
- Cambiar separadores decimales/miles si están mezclados.

---

## 3) Normalización de texto (15 min)

Aquí buscamos consistencia para que `UNIQUE` y filtros funcionen bien.

### 3.1 TRIM (ESPACIOS)
**Problema:** espacios al inicio/fin o repetidos dentro.
- Función: `TRIM(texto)` / `ESPACIOS(texto)` (según idioma del Sheet)

**Uso recomendado (sin concatenar):**
- Crea una **columna nueva** temporal: `*_clean`
- Aplica `TRIM` solo.
- Copia y pega valores (Pegado especial → solo valores) si vas a reemplazar la original.

### 3.2 LOWER / UPPER
**Objetivo:** estandarizar *cuando el caso no importa* (ej. códigos, tags).
- `LOWER(texto)`
- `UPPER(texto)`

### 3.3 NOMBREPROPIO (PROPER)
**Objetivo:** nombres y títulos más legibles.
- `NOMBREPROPIO(texto)` / `PROPER(texto)`

> Regla simple: **TRIM primero**, luego caso (`LOWER/UPPER/PROPER`).  
Como no concatenamos, lo hacemos en **pasos** con columnas auxiliares.

---

## 4) Verificación rápida con funciones básicas (10 min)

Estas funciones son tu “control de calidad” inmediato.

### 4.1 COUNT
- `COUNT(rango)` cuenta **números**
- `COUNTA(rango)` cuenta **no vacíos** (si la usas, explícala)

**Uso DA:** detectar si una columna “numérica” está entrando como texto:
- Si `COUNT(columna)` da muy bajo, probablemente hay texto/símbolos.

### 4.2 UNIQUE
- `UNIQUE(rango)`

**Uso DA:** revisar cuántas categorías reales hay (y si hay variantes por mayúsculas/espacios).

### 4.3 SUM / AVERAGE
- `SUM(rango)`
- `AVERAGE(rango)`

**Uso DA:** validar que la columna ya se comporta como número.

### 4.4 SUMIF
- `SUMIF(rango_criterio, criterio, rango_suma)`

**Uso DA:** validar agregaciones simples por categoría (ej. por álbum, por era, etc.).

---

## 5) Limpieza guiada: columna por columna (35 min)

**Meta del bloque:** avanzar con una rutina repetible y documentada.

> Importante: como el dataset puede variar, piensa en esta sección como un **guion**: identifica el problema real en tu archivo y aplica el patrón.

### 5.1 Rutina DA por columna (siempre igual)
1. **Inspecciona** 20–30 filas (visualmente) + mira el tipo.
2. **Detecta el issue** (ej. espacios, mayúsculas, símbolos, separador).
3. **Define la acción mínima** (Buscar/Reemplazar o 1 función simple).
4. **Aplica** en `*_work`.
5. **Verifica** con `COUNT/UNIQUE/SUM/AVERAGE`.
6. **Registra** en `change_log` con ejemplo antes/después.

---

## 5.2 Patrones de limpieza (elige según lo que encuentres)

### A) Texto con espacios “invisibles”
- Acción: nueva columna `col_clean` con `TRIM(col_original)`
- Verificación: `UNIQUE(col_original)` vs `UNIQUE(col_clean)` (debería bajar)

### B) Categorías con diferentes mayúsculas
- Acción: `NOMBREPROPIO` o `LOWER` (según el caso de uso)
- Verificación: `UNIQUE` antes/después

### C) Separadores inconsistentes en listas
Ej.: `Writer1, Writer2 / Writer3` → `Writer1; Writer2; Writer3`
- Acción: **Buscar y Reemplazar**
  - Reemplaza `", "` por `"; "` (si aplica)
  - Reemplaza `" / "` por `"; "`
- Verificación: revisa 10 filas manualmente + `UNIQUE` (si es categoría)

### D) Números que llegaron como texto (símbolos)
Ej.: `$1,234` o `45%`
- Acción: **Buscar y Reemplazar** para eliminar `$` y `%`
- Acción (si hay comas de miles): reemplaza `,` por vacío **solo si** es separador de miles (ojo con decimales)
- Verificación: `COUNT(rango)` debe subir; `SUM/AVERAGE` deben funcionar

---

## 5.3 Guion sugerido por hojas

### Hoja: `songs_work`
Recorre columnas en este orden:
1. **Identificadores / texto corto** (título, artista, álbum, era)
2. **Columnas de listas** (writers, producers, genres, etc. si existen)
3. **Métricas numéricas** (streams, duration, bpm, chart positions, etc. si existen)
4. **Fechas** (si existen): estandariza formato (sin fórmulas complejas; solo consistencia visual + validación)

### Hoja: `albums_work`
Recorre:
1. Nombre de álbum / era
2. Campos categóricos
3. Métricas numéricas
4. Fechas

> El orden importa: primero lo que afecta categorías (para poder agrupar), luego números (para poder calcular).

---

## 6) Documentación: cómo llenar el change log (10 min)

Cada cambio debe quedar rastreable. Ejemplo de registro:

- **timestamp:** 2026-02-08 10:35
- **sheet:** songs_work
- **column:** writers
- **issue:** separadores inconsistentes (coma y slash)
- **action:** Buscar y Reemplazar: `", "` → `"; "` y `" / "` → `"; "`
- **before_example:** `A. Smith, B. Doe / C. Lee`
- **after_example:** `A. Smith; B. Doe; C. Lee`
- **owner:** tu_nombre

> Regla DA: si no está en el log, “no pasó”.

---

## Cierre de la sesión (10 min)

### Takeaways (lo que te debes llevar hoy)
1. La limpieza mínima es una **habilidad de impacto**: hace que el dataset sea usable *ya*.
2. **Buscar y Reemplazar** es tu primera herramienta para estandarizar texto y símbolos sin fórmulas.
3. Evitar fórmulas concatenadas te obliga a trabajar en **pasos claros** (y eso mejora trazabilidad).
4. Verificar con `COUNT`, `UNIQUE`, `SUM`, `AVERAGE`, `SUMIF` evita errores silenciosos.
5. El `change_log` convierte tu limpieza en un trabajo **reproducible y auditable**.

### Checklist de salida (para que la práctica funcione)
- [ ] `songs_work` y `albums_work` existen y son copias del raw
- [ ] Cambios registrados en `change_log` (mínimo 5 entradas)
- [ ] Al menos 1 columna numérica ya permite `SUM` y `AVERAGE`
- [ ] Al menos 1 columna categórica reduce variantes tras `TRIM` / `NOMBREPROPIO`
- [ ] Una mini-validación con `SUMIF` funciona (una categoría → un total)

### Próxima sesión (práctica)
Aplicaremos este guion en vivo y construiremos un `summary_template` con 2–3 insights simples.

---